## 1) Raw File Header Handling
NASA POWER hourly exports include a metadata header before the tabular dataset.
The preprocessing script scans the raw CSV and locates the first line starting
with `YEAR,`, which marks the beginning of the data table. All preceding lines
are treated as metadata and skipped.
The table itself is comma-separated.

In [67]:
from pathlib import Path
import pandas as pd

raw_path = Path("Data/Dataset_phase_1.csv")

# Step 1 — Find the line where the data table starts
with raw_path.open("r", encoding="utf-8", errors="ignore") as f:
    for i, line in enumerate(f):
        if line.lstrip().startswith("YEAR,"):
            table_start = i
            break
    else:
        raise ValueError("Could not find the data table header starting with 'YEAR,'")

print("Table starts at line:", table_start)

# Step 2 — Read the table from that line onward (comma-separated)
df = pd.read_csv(raw_path, sep=",", skiprows=table_start)

print(df.head(2))
print(df.columns.tolist())

Table starts at line: 10
   YEAR  MO  DY  HR  ALLSKY_SFC_SW_DWN    T2M
0  2021   1   1   0                0.0  23.35
1  2021   1   1   1                0.0  22.70
['YEAR', 'MO', 'DY', 'HR', 'ALLSKY_SFC_SW_DWN', 'T2M']


## 2) Rename columns (keep units honest)

In [68]:
df = df.rename(columns={
    "YEAR": "year",
    "MO": "month",
    "DY": "day",
    "HR": "hour",
    "ALLSKY_SFC_SW_DWN": "ghi_wh_m2",
    "T2M": "temp_c"
})

print(df.columns.tolist())

['year', 'month', 'day', 'hour', 'ghi_wh_m2', 'temp_c']


## 3) Creating timestamp

In [69]:
df["time"] = pd.to_datetime(df[["year","month","day"]]) \
             + pd.to_timedelta(df["hour"], unit="h")

df = df.sort_values("time")
print(df.head())

   year  month  day  hour  ghi_wh_m2  temp_c                time
0  2021      1    1     0        0.0   23.35 2021-01-01 00:00:00
1  2021      1    1     1        0.0   22.70 2021-01-01 01:00:00
2  2021      1    1     2        0.0   22.03 2021-01-01 02:00:00
3  2021      1    1     3        0.0   21.40 2021-01-01 03:00:00
4  2021      1    1     4        0.0   20.78 2021-01-01 04:00:00


## 4) Confirm Temporal Resolution (Hourly)

In [70]:
# Temporal resolution check
temporal_step = df["time"].diff().mode()[0]

print("Most common temporal step:", temporal_step)

Most common temporal step: 0 days 01:00:00


## 5) Missing Data Percentage (Values)

In [71]:
# replace -999 (NASA POWER uses -999 for missing values
df["ghi_wh_m2"] = df["ghi_wh_m2"].replace(-999, pd.NA)
df["temp_c"] = df["temp_c"].replace(-999, pd.NA)

# computing missing values
missing_ghi_pct = df["ghi_wh_m2"].isna().mean() * 100
missing_temp_pct = df["temp_c"].isna().mean() * 100

print(f"Missing GHI values: {missing_ghi_pct:.3f}%")
print(f"Missing Temperature values: {missing_temp_pct:.3f}%")

Missing GHI values: 1.752%
Missing Temperature values: 0.000%


## 6) Missing Timestamps (Gaps)

In [74]:
full_index = pd.date_range(
    start=df["time"].min(),
    end=df["time"].max(),
    freq="h"
)

missing_ts_pct = (len(full_index) - len(df)) / len(full_index) * 100

print(f"Missing hourly timestamps: {missing_ts_pct:.3f}%")
print("Rows:", len(df))

Missing hourly timestamps: 0.000%
Rows: 43824


## 7) Drop missing rows with 1.75% missing GHI and save clean dataset

In [75]:
import pandas as pd

# 1. Drop missing GHI values (baseline Phase 1 choice)
df_clean = df.dropna(subset=["ghi_wh_m2"]).copy()

# 2. Keep only the useful columns
df_clean = df_clean[["time", "ghi_wh_m2", "temp_c"]]

# 3. Sort chronologically
df_clean = df_clean.sort_values("time")

# 4. Save in the SAME folder (current directory)
df_clean.to_csv("Data/Dataset_phase_1_clean.csv", index=False)

print("Saved cleaned dataset as: Dataset_phase_1_clean.csv")
print("Rows:", len(df_clean))


Saved cleaned dataset as: Dataset_phase_1_clean.csv
Rows: 43056
